### Conversation Q&A Chatbot
In many Q&A applications we want to allow the user to have a back-and-forth conversation, meaning the application needs some sort of "memory" of past questions and answers, and some logic for incorporating those into its current thinking.

In this guide we focus on adding logic for incorporating historical messages. Further details on chat history management is covered in the previous videos.

We will cover two approaches:

- Chains, in which we always execute a retrieval step;
- Agents, in which we give an LLM discretion over whether and how to execute a retrieval step (or multiple steps).

In [ ]:
!pip install -U langchain-groq
!pip install python-dotenv
!pip install -U -q langchain langchain_community pypdf chromadb PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
!!pip install -U -q sentence-transformers

['\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/227.1 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b227.1/227.1 kB\x1b \x1b7.3 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/21.3 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b3.6/21.3 MB\x1b \x1b107.8 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b9.3/21.3 MB\x1b \x1b135.1 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━\x1b \x1b16.1/21.3 MB\x1b \x1b180.2 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b21.3/21.3 MB\x1b \x1b216.3 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b21

In [ ]:
import os
from google.colab import userdata
from langchain_groq import ChatGroq


# Retrieve the secret key
groq_api_key = userdata.get('GROQ_API_KEY')

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c21e39e8d30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c21e39e8070>, model_name='Llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [ ]:
from google.colab import userdata

# Retrieve the secret key
my_secret_key = userdata.get('GROQ_API_KEY')

# Use the key as needed
if my_secret_key:
    print("API key loaded successfully!")
else:
    print("Failed to load API key.")

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [ ]:

loader=PyPDFDirectoryLoader('/content/drive/MyDrive/Ashal Assistant/source_documents')
docs=loader.load()

In [ ]:
docs[0]

Document(metadata={'source': '/content/drive/MyDrive/Ashal Assistant/source_documents/07-Islamic-Adab-Good-Manners-or-Etiquette.pdf', 'page': 0}, page_content=' \nChildren’s Education Series \nBook - 7 \n \n \n \n \n \n \n \n \n \n(Good Manners/Etiquette) \n  \n \n \n \nAmir Zaman \nNazma Zaman \n')

In [ ]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain_community.vectorstores import Chroma

In [ ]:
#from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"                              ##"sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7c20bf9e6680>)

In [ ]:
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
response=rag_chain.invoke({"input":"For any kind of promise what should we say?"})
response

{'input': 'For any kind of promise what should we say?',
 'context': [Document(metadata={'page': 6, 'source': '/content/drive/MyDrive/Ashal Assistant/source_documents/07-Islamic-Adab-Good-Manners-or-Etiquette.pdf'}, page_content='7 While entering your own house or  \napartment, say: Bismillah . \n(I am beginning (entering)it in the \nname of Allah)  \n \n \nWhen a good news is heard, or gratitude to \nAllah is to be expressed for any reason, \n(after eating dinner or lunch, etc.,) say:  \nAlhamdulillah \n(All praises belong to Allah alone)  \n \n \n \nWhen someone departs from you; say:   \nFi-amanillah  (I place you to the safety \nand protection of Allah)'),
  Document(metadata={'page': 9, 'source': '/content/drive/MyDrive/Ashal Assistant/source_documents/07-Islamic-Adab-Good-Manners-or-Etiquette.pdf'}, page_content='10   \nWhile in pain, say: \nYa Allah  (O Allah) \n \n \n \nFor any promise or for intending to do \nanything in future, say: Insha Allah \n(If Allah wishes) \n \n \nIf 

In [ ]:
response['answer']

'For any promise or for intending to do anything in the future, you should say "Insha Allah" (If Allah wishes).'

In [ ]:
rag_chain.invoke({"input":"tell me more about it?"})

{'input': 'tell me more about it?',
 'context': [Document(metadata={'page': 24, 'source': '/content/drive/MyDrive/Ashal Assistant/source_documents/07-Islamic-Adab-Good-Manners-or-Etiquette.pdf'}, page_content='25   \nPut garbage inside the correct bins. \n \n \nNever to underestimate any person without \nknowing him well. Never to form a negative \nopinion about any person on the basis of \nothers’ adverse report before verifying the \ntruth. Not to take a hasty decision without \nproper verification of the relevant facts and \nfigures. Not to pry into other’s privacy.'),
  Document(metadata={'page': 2, 'source': '/content/drive/MyDrive/Ashal Assistant/source_documents/07-Islamic-Adab-Good-Manners-or-Etiquette.pdf'}, page_content='3  \n“Assalamu-Alaikum” Peace be upon you \n \n \n \nPublished by \nInstitute of Social Engineering, Canada \nwww.themessagecanada.com 1st  Edition: July 2012 \nPrice: $ 2 Amir Zaman  \nNazma Zaman  \nEmail: amiraway@hotmail.com  \nwww.themessagecanada.com'),

### Adding Chat History

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7c20bf9e6680>))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just refor

In [ ]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [ ]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="For any kind of promise what should we say?"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Tell me more about it?"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

For any promise, you should say "Insha Allah" (If Allah wishes).


In [ ]:
chat_history

[HumanMessage(content='For any kind of promise what should we say?'),
 AIMessage(content='For any promise or intending to do something in the future, you should say "Insha Allah" which means "If Allah wishes".')]

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "which hand we should use while giving?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'According to the context, while giving a gift, you should use your right hand.'

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"I don't know. The text doesn't mention specific ways of giving or receiving gifts."

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc1213"}},
)["answer"]

'I\'m assuming you\'re referring to Islamic practices mentioned in the context. Here are a few common ways mentioned:\n\n* Saying "Bismillah" before taking food or drink with the right hand.\n* Eating from the front side of the plate, not the middle or other side.\n* Offering Salam (peace be upon you) in various situations, such as entering an empty masjid, before going to bed, when answering a phone, etc.\n* Using the right hand to take food or drink, and the left hand for cleaning after using the toilet.'